# To run server: python llava_server.py
# To run client: python post_imgtxt.py

pip install fastapi uvicorn
pip install accelerate

In [ ]:
!pip install fastapi uvicorn
!pip install accelerate
!pip install pillow

In [ ]:
from PIL import Image
from fastapi import FastAPI, File, Form, UploadFile, HTTPException
from fastapi.responses import Response
import requests
import uvicorn
import logging
import os

In [ ]:
import torch
print(torch.cuda.is_available())

True


In [ ]:
import torch
print(torch.__version__)

2.3.1+cu121


In [ ]:
!pip install torch --upgrade

In [ ]:
!pip install flash_attn

In [ ]:
import torch
from transformers import AutoProcessor, AutoModelForCausalLM

# 檢查 CUDA 可用性
if not torch.cuda.is_available():
    raise EnvironmentError("CUDA is not available. Please check your GPU settings.")

model_id = "microsoft/Phi-3-vision-128k-instruct"

# 嘗試加載模型
try:
    model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", trust_remote_code=True, torch_dtype="auto")
    processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)
except Exception as e:
    print(f"Error loading model: {e}")
    raise

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
!pip install torch --upgrade
!pip install transformers --upgrade
!pip install accelerate
!pip install pillow

In [ ]:
from pydantic import BaseModel

In [ ]:
from transformers import AutoProcessor, AutoModelForCausalLM
import torch

In [ ]:
model_id = "microsoft/Phi-3-vision-128k-instruct"

In [ ]:
!pip install accelerate

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda", trust_remote_code=True, torch_dtype="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
generation_args = {
    "max_new_tokens": 500,
    "temperature": 0.0,
    "do_sample": False,
}

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
app = FastAPI()

In [ ]:
@app.get("/")
def home():
    return Response("Hello World")

In [ ]:
@app.post("/imgtxt")
def post_imgtxt(image: UploadFile = File(...), text: str = Form(...)):
    print(image.filename)
    print(text)
    with open(image.filename, "wb") as f:
        content = image.file.read()
        f.write(content)
    f.close()

    # VLM = phi-3-vision
    print(image.filename)
    img = Image.open(image.filename)
    inst = ", please answer in a simple sentence"
    messages = [
        {"role": "user", "content": "<|image_1|>\n"+text+inst}, # "What is shown in this image?"},
    ]
    prompt = processor.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = processor(prompt, [img], return_tensors="pt").to("cuda:0")
    generate_ids = model.generate(**inputs, eos_token_id=processor.tokenizer.eos_token_id, **generation_args) #max_new_tokens=200

    # remove input tokens
    outputs = generate_ids[:, inputs['input_ids'].shape[1]:]
    result = processor.batch_decode(outputs, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    print(result)
    return Response(result)

In [ ]:
if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=5000, log_level="info")

INFO:     Started server process [10682]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:5000 (Press CTRL+C to quit)
